In [1]:
import heapq
import numpy as np
from collections import deque

In [2]:
def imprimir_sudoku(matriz):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 21)
        for j in range(9):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            print(matriz[i][j], end=" ")
        print()
        if i == 8:
            print()

In [3]:
def verificar_objetivo(matriz):
    for linha in matriz:
        if len(np.unique(linha)) != 9 or 0 in linha:
            return False

    for coluna in range(9):
        if len(np.unique(matriz[:, coluna])) != 9:
            return False

    for i in range(0, 9, 3):
        for j in range(0, 9, 3):
            quadrado = matriz[i:i+3, j:j+3].flatten()
            if len(np.unique(quadrado)) != 9:
                return False

    return True

def gerar_sucessores(matriz):
    sucessores = []

    # Encontra a primeira célula vazia
    for i in range(9):
        for j in range(9):
            if matriz[i][j] == 0:
                # Preenche a célula vazia com números válidos
                for num in range(1, 10):
                    if validar_atribuicao(matriz, i, j, num):
                        novo_estado = np.copy(matriz)
                        novo_estado[i][j] = num
                        sucessores.append(novo_estado)
                # Como só uma célula foi preenchida, podemos retornar os sucessores
                return sucessores

    # Se não houver células vazias, retorna uma lista vazia
    return sucessores

def validar_atribuicao(matriz, linha, coluna, num):
    if num in matriz[linha]:
        return False

    if num in matriz[:, coluna]:
        return False

    inicio_linha, inicio_coluna = 3 * (linha // 3), 3 * (coluna // 3)
    quadrado = matriz[inicio_linha:inicio_linha+3, inicio_coluna:inicio_coluna+3]
    if num in quadrado:
        return False

    return True


In [4]:
def bfs(sudoku_inicial):
    fila = deque()
    num_visitados = 0

    fila.append(sudoku_inicial)

    while fila:
        estado = fila.popleft()
        num_visitados += 1

        if verificar_objetivo(estado):
            return estado, num_visitados

        sucessores = gerar_sucessores(estado)

        for suc in sucessores:
            fila.append(suc)  # Adiciona o sucessor à fila de estados a serem explorados
    return None, num_visitados

In [5]:
""" def dfs(sudoku, profundidade_maxima):
    # Implementação da busca em profundidade (DFS)
    def dfs_recursivo(estado, profundidade):
        if profundidade > profundidade_maxima:
            return None, 0  # Retorna None se a profundidade máxima foi atingida

        if verificar_objetivo(estado):
            return estado, 1  # Retorna o estado se for o objetivo

        sucessores = gerar_sucessores(estado)
        num_visitados = 1  # Contador de estados visitados

        for suc in sucessores:
            solucao, visitados = dfs_recursivo(suc, profundidade + 1)
            num_visitados += visitados  # Atualiza o contador de estados visitados
            if solucao is not None:
                return solucao, num_visitados  # Retorna a solução se encontrada

        return None, num_visitados  # Retorna None se não encontrar uma solução nesta ramificação

    # Chamada inicial da função DFS
    return dfs_recursivo(sudoku, 0)


def ids(sudoku_inicial):
    profundidade_maxima = 1
    print("Profundidade:", profundidade_maxima)

    while True:
        solucao, num_visitados = dfs(sudoku_inicial, profundidade_maxima)
        if solucao is not None:
            return solucao, num_visitados
        profundidade_maxima += 1 """

' def dfs(sudoku, profundidade_maxima):\n    # Implementação da busca em profundidade (DFS)\n    def dfs_recursivo(estado, profundidade):\n        if profundidade > profundidade_maxima:\n            return None, 0  # Retorna None se a profundidade máxima foi atingida\n\n        if verificar_objetivo(estado):\n            return estado, 1  # Retorna o estado se for o objetivo\n\n        sucessores = gerar_sucessores(estado)\n        num_visitados = 1  # Contador de estados visitados\n\n        for suc in sucessores:\n            solucao, visitados = dfs_recursivo(suc, profundidade + 1)\n            num_visitados += visitados  # Atualiza o contador de estados visitados\n            if solucao is not None:\n                return solucao, num_visitados  # Retorna a solução se encontrada\n\n        return None, num_visitados  # Retorna None se não encontrar uma solução nesta ramificação\n\n    # Chamada inicial da função DFS\n    return dfs_recursivo(sudoku, 0)\n\n\ndef ids(sudoku_inicial)

In [6]:
def dfs_limitado(sudoku, profundidade_maxima):
    # Implementação da busca em profundidade (DFS) com limite de profundidade máxima
    def dfs_recursivo(estado, profundidade):
        if profundidade > profundidade_maxima:
            return None, 0  # Retorna None se a profundidade máxima foi atingida

        if verificar_objetivo(estado):
            return estado, 1  # Retorna o estado se for o objetivo

        sucessores = gerar_sucessores(estado)
        num_visitados = 1  # Contador de estados visitados

        for suc in sucessores:
            solucao, visitados = dfs_recursivo(suc, profundidade + 1)
            num_visitados += visitados  # Atualiza o contador de estados visitados
            if solucao is not None:
                return solucao, num_visitados  # Retorna a solução se encontrada

        return None, num_visitados  # Retorna None se não encontrar uma solução nesta ramificação

    # Chamada inicial da função DFS
    return dfs_recursivo(sudoku, 0)

In [7]:
def ids(sudoku_inicial):
    profundidade_maxima = 1
    num_visitados=0
    print("Profundidade:", profundidade_maxima)

    while True:
        solucao, num_visitados_lim = dfs_limitado(sudoku_inicial, profundidade_maxima)
        num_visitados+=num_visitados_lim
        if solucao is not None:
            return solucao, num_visitados
        else:
            profundidade_maxima += 1
            print(profundidade_maxima)

In [8]:
def uniform_cost_search(sudoku_inicial):
    fila = [(0, sudoku_inicial.tobytes())]  # Fila de prioridade inicialmente com o estado inicial e custo zero
    visitados = 0  # Contador de estados visitados

    while fila:
        custo, estado = heapq.heappop(fila)  # Remove e retorna o estado com menor custo da fila
        estado = np.frombuffer(estado, dtype=int).reshape((9, 9))  # Convertendo a string de volta para a matriz
        visitados += 1  # Incrementa o contador de estados visitados

        if verificar_objetivo(estado):
            return estado, visitados  # Retorna o estado e o número de estados visitados se for o objetivo

        sucessores = gerar_sucessores(estado)

        for suc in sucessores:
            estado_str = suc.tobytes()  # Convertendo a matriz em uma string
            heapq.heappush(fila, (np.count_nonzero(suc == 0), estado_str))  # Adiciona o sucessor à fila com seu custo


    return None, visitados

In [9]:
import numpy as np
import heapq

def calcular_heuristica(matriz):
    heuristica = np.zeros_like(matriz, dtype=int)  

    for i in range(9):
        for j in range(9):
            if matriz[i][j] == 0:
                valores_possiveis = set(range(1, 10)) - set(matriz[i]) - set(matriz[:, j]) - set(matriz[i//3*3:i//3*3+3, j//3*3:j//3*3+3].flatten())
                heuristica[i][j] = len(valores_possiveis)

    return heuristica

def buscar_proxima_jogada(matriz, heuristica):
    menor_heuristica = float('inf')
    melhor_jogada = None

    for i in range(9):
        for j in range(9):
            if matriz[i][j] == 0 and heuristica[i][j] < menor_heuristica:
                menor_heuristica = heuristica[i][j]
                melhor_jogada = (i, j)

    return melhor_jogada

def resolver_gulosa(sudoku):
    visitados = 0
    heap = []  # Usaremos um heap para a fila de prioridade

    # Calcula a heurística inicial e insere na fila de prioridade
    heapq.heappush(heap, (0, sudoku.tobytes(), calcular_heuristica(sudoku)))

    while heap:
        _, sudoku, heuristica = heapq.heappop(heap)  # Prioridade é o número de células em branco
        sudoku = np.frombuffer(sudoku, dtype=int).reshape((9, 9))
        visitados += 1

        proxima_jogada = buscar_proxima_jogada(sudoku, heuristica)
        
        if proxima_jogada is None:
            return sudoku, visitados
        else:
            i, j = proxima_jogada
            valores_possiveis = set(range(1, 10)) - set(sudoku[i]) - set(sudoku[:, j]) - set(sudoku[i//3*3:i//3*3+3, j//3*3:j//3*3+3].flatten())

            if len(valores_possiveis) == 0:
                continue
            else:
                for valor in valores_possiveis:
                    novo_sudoku = sudoku.copy()
                    novo_sudoku[i][j] = valor
                    novo_sudoku_str=novo_sudoku.tobytes()
                    heapq.heappush(heap, (np.count_nonzero(novo_sudoku == 0), novo_sudoku_str, calcular_heuristica(novo_sudoku)))  # Atualiza a fila de prioridade com a nova configuração

    return sudoku, visitados


In [13]:
import numpy as np
import heapq

def heuristica_manhattan(matriz):
    heuristica = np.zeros_like(matriz, dtype=int)
    for i in range(9):
        for j in range(9):
            if matriz[i][j] == 0:
                valores_possiveis = set(range(1, 10)) - set(matriz[i]) - set(matriz[:, j]) - set(matriz[i//3*3:i//3*3+3, j//3*3:j//3*3+3].flatten())
                heuristica[i][j] = len(valores_possiveis) + manhattan_distancia(i, j)
            else:
                heuristica[i][j] = 0
    return heuristica

def manhattan_distancia(i, j):
    centro_linha, centro_coluna = 4, 4
    return abs(centro_linha - i) + abs(centro_coluna - j)



def buscar_proxima_jogada_Astar(matriz, heuristica):
    menor_custo_total = float('inf')
    melhor_jogada = None

    for i in range(9):
        for j in range(9):
            if matriz[i][j] == 0 and heuristica[i][j] < menor_custo_total:
                menor_custo_total = heuristica[i][j]
                melhor_jogada = (i, j)

    return melhor_jogada

def resolver_Astar(sudoku):
    visitados = 0
    heap = []  

    heapq.heappush(heap, (0, 0, sudoku.tobytes(), heuristica_manhattan(sudoku)))  # Adicionando o custo acumulado como prioridade

    while heap:
        _, custo_acumulado, sudoku, heuristica = heapq.heappop(heap)  
        sudoku = np.frombuffer(sudoku, dtype=int).reshape((9, 9))
        visitados += 1

        proxima_jogada = buscar_proxima_jogada_Astar(sudoku, heuristica)
        
        if proxima_jogada is None:
            return sudoku, visitados
        else:
            i, j = proxima_jogada
            valores_possiveis = set(range(1, 10)) - set(sudoku[i]) - set(sudoku[:, j]) - set(sudoku[i//3*3:i//3*3+3, j//3*3:j//3*3+3].flatten())

            if len(valores_possiveis) == 0:
                continue
            else:
                for valor in valores_possiveis:
                    novo_sudoku = sudoku.copy()
                    novo_sudoku[i][j] = valor
                    novo_custo_acumulado = custo_acumulado
                    novo_sudoku_str = novo_sudoku.tobytes()
                    heapq.heappush(heap, (novo_custo_acumulado + np.count_nonzero(novo_sudoku == 0), novo_custo_acumulado, novo_sudoku_str, heuristica_manhattan(novo_sudoku)))  

    return sudoku, visitados


In [18]:
sudoku_str = "800000000 003600000 070090200 050007000 000045700 000100030 001000068 008500010 090000400"
sudoku_str = sudoku_str.replace(" ", "")
sudoku_inicial = np.array([int(c) for c in sudoku_str]).reshape((9, 9))

In [19]:
#solucao = bfs(sudoku_inicial)
#solucao = ids(sudoku_inicial)
#solucao = uniform_cost_search(sudoku_inicial)
#solucao=resolver_gulosa(sudoku_inicial)
solucao=resolver_Astar(sudoku_inicial)

if solucao[0] is not None:    
    print("Solução encontrada:")
    imprimir_sudoku(solucao[0])
    print("Numero de estados visitados: ", solucao[1])
else:
    print("Não foi encontrada uma solução.")


Solução encontrada:
8 1 2 | 7 5 3 | 6 4 9 
9 4 3 | 6 8 2 | 1 7 5 
6 7 5 | 4 9 1 | 2 8 3 
---------------------
1 5 4 | 2 3 7 | 8 9 6 
3 6 9 | 8 4 5 | 7 2 1 
2 8 7 | 1 6 9 | 5 3 4 
---------------------
5 2 1 | 9 7 4 | 3 6 8 
4 3 8 | 5 2 6 | 9 1 7 
7 9 6 | 3 1 8 | 4 5 2 

Numero de estados visitados:  297141
